In [1]:
#data manipulation
import numpy as np
import pandas as pd

from datetime import datetime
import itertools

#data visualization
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.model_selection import TimeSeriesSplit

#ignore warnings
import warnings
warnings.filterwarnings("ignore")

import acquire
import prepare_2

<class 'dict'>

The API accepts GET requests for all endpoints, where endpoints are prefixed
with

    /api/{version}

Where version is "v1"

Valid endpoints:

- /stores[/{store_id}]
- /items[/{item_id}]
- /sales[/{sale_id}]

All endpoints accept a `page` parameter that can be used to navigate through
the results.

max_page: 3
next_page: /api/v1/items?page=2
max_page: 3
next_page: /api/v1/items?page=3
max_page: 3
next_page: None
max_page: 1
next_page: None
max_page: 183
next_page: /api/v1/sales?page=2


## Exploration

### Acquire:

In [7]:
df = acquire.get_all_the_data()

(913000, 14)

In [8]:
df = prepare_2.prep_store_data(df)